# Sample Notebook 2 for Picasso
This notebook shows some basic interaction with the ``picasso`` library. It assumes to have a working picasso installation, for example, from PyPI (see: https://github.com/jungmannlab/picasso?tab=readme-ov-file#via-pypi).

The sample data was created using Picasso:Simulate.

## Load Localizations

In [1]:
from picasso import io
path = 'data/testdata_locs.hdf5'
locs, info = io.load_locs(path)

print(f'Loaded {len(locs)} locs.')

Loaded 564 locs.


## Info file
The info file is now a list of dictionaries. Each step in picasso adds an element to the list.

In [2]:
for i in range(len(info)):
    print(info[i]['Generated by'])
    
# extract width and height:
width, height = info[0]['Width'], info[0]['Height']
print(f'Image height: {height}, width: {width}')

Picasso simulate
Picasso Localize
Image height: 32, width: 32


## Filter localizations

Filter localizations, i.e., via sx and sy: Remove all localizations that are not within a circle around a center position.

In [3]:
sx_center = 0.82
sy_center = 0.82

radius = 0.04 

to_keep = (locs.sx-sx_center)**2 + (locs.sy-sy_center)**2 < radius**2

filtered_locs = locs[to_keep]
print(f'Length of locs before filtering {len(locs)}, after filtering {len(filtered_locs)}.')

Length of locs before filtering 564, after filtering 480.


## Saving localizations
Add new info to the yaml file and save everything.

In [4]:
import os.path as ospath
# Create a new dictionary for the new info
new_info = {}
new_info["Generated by"] = "Picasso Jupyter Notebook"
new_info["Filtered"] = 'Circle'
new_info["sx_center"] = sx_center
new_info["sy_center"] = sy_center
new_info["radius"] = radius

info.append(new_info)
new_path = path.replace('.hdf5', '_sample_notebook2.hdf5')

io.save_locs(new_path, filtered_locs, info)

print(f'{len(filtered_locs)} locs saved to {new_path}.')

480 locs saved to data/testdata_locs_sample_notebook2.hdf5.


## Manually export images
Use the picasso functions to render images.

In [5]:
# Get minimum / maximum localizations to define the ROI to be rendered 
import numpy as np
from picasso import render
import matplotlib.pyplot as plt

x_min = 0
x_max = width
y_min = 0
y_max = height

viewport =  (y_min, x_min), (y_max, x_max)
oversampling = 100  # number of display pixels per camera pixel
len_x, image = render.render(locs, viewport=viewport, oversampling=oversampling, blur_method='smooth')
plt.imsave('data/test.png', image, cmap='hot', vmax=1)

# Custom ROI with higher oversampling
viewport =  (5, 15), (6, 16)
oversampling = 500  # number of display pixels per camera pixel
len_x, image = render.render(locs, viewport = viewport, oversampling=oversampling, blur_method='smooth')
plt.imsave('data/test_zoom.png', image, cmap='hot')

## Calculate kinetics
Use the picasso functions to calculate kinetics.

In [6]:
from picasso import postprocess

# Note: to calculate dark times you need picked localizations of single binding sites
path = 'data/testdata_locs_picked_single.hdf5'
picked_locs, info = io.load_locs(path)

# Link localizations and calculate dark times
linked_locs = postprocess.link(picked_locs, info, r_max=0.05, max_dark_time=1)
linked_locs_dark = postprocess.compute_dark_times(linked_locs)

print('Average bright time {:.2f} frames'.format(np.mean(linked_locs_dark.n)))
print('Average dark time {:.2f} frames'.format(np.mean(linked_locs_dark.dark)))

# Compare with simulation settings:
integration_time = info[0]['Camera.Integration Time']
tau_b = info[0]['PAINT.taub']
k_on = info[0]['PAINT.k_on']
imager = info[0]['PAINT.imager']
tau_d = 1/(k_on*imager)*10**9*1000

print('------')

print('ON Measured {:.2f} ms \t Simulated {:.2f} ms'.format(np.mean(linked_locs_dark.n)*integration_time, tau_b))
print('OFF Measured {:.2f} ms \t Simulated {:.2f} ms'.format(np.mean(linked_locs_dark.dark)*integration_time, tau_d))
print('\nSome error is expected due to low localization count per site!')

Average bright time 2.50 frames
Average dark time 213.87 frames
------
ON Measured 750.00 ms 	 Simulated 500.00 ms
OFF Measured 64160.00 ms 	 Simulated 125000.00 ms

Some error is expected due to low localization count per site!
